In [1]:
import csv
import numpy as np
import pandas as pd
import os
import re
from dotenv import load_dotenv
from openai import OpenAI
from datasets import load_dataset
import openai
load_dotenv()
openai.api_key = os.environ.get("REASONING_API_KEY")
if not openai.api_key:
    raise ValueError("API Key for OpenAI not set!")
client = OpenAI(api_key=os.environ.get("REASONING_API_KEY"))

/Users/vivekvajipey/miniconda3/envs/reasoning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def query_gpt(prompt, temp):
    message = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{prompt}"}

]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages = message,
        max_tokens=1024,
        temperature=temp,
        n=1,
        stop=None,
    )

    return response.choices[0].message.content

In [9]:
import ast
df = pd.read_csv('data/112_gsm8k_gpt35_cot_responses_autograded.csv')
df['CoT Steps'] = df['CoT Steps'].apply(ast.literal_eval)
df['Paraphrased CoT Steps'] = df['Paraphrased CoT Steps'].apply(ast.literal_eval)
df.head(3)

,Problem Number,Question,Answer,CoT Response,CoT Sentences,CoT Steps,split,Paraphrased Question,Paraphrased CoT Steps,True Answer,Final Score
0,0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,"First, let's find out how many clips Natalia s...","**\nFirst, let's find out how many clips Natal...","[First, let's find out how many clips Natalia ...",train,Derek collected seashells from 48 of his frien...,"[First, we need to figure out the number of se...",72,1
1,1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,"To find out how much Weng earned, we need to c...","**\nTo find out how much Weng earned, we need ...","[To find out how much Weng earned, we need to ...",train,Liam makes $12 per hour tutoring. He tutored f...,"[First, we need to figure out Liam's earnings ...",10,1
2,2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<...","First, let's calculate how much money Betty ha...","**\nFirst, let's calculate how much money Bett...","[First, let's calculate how much money Betty h...",train,Oliver is collecting stickers for his album wh...,"[First, we need to figure out how many sticker...",5,1


In [8]:
prompt = f"Q: {df.Question.tolist()[0]}\nLet's think step by step.\nA:"
prompt 

"Q:Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\nLet's think step by step.\nA:"

In [11]:
message = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{prompt}"}
]

In [12]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages = message,
        max_tokens=1024,
        temperature=0.0,
        n=1,
        stop=None,
        logprobs=True,
        top_logprobs=5
    )

In [20]:
response.choices[0].message.content == df['CoT Response'].tolist()[0]

True

In [40]:
# response.choices[0].logprobs.content
len(response.choices[0].logprobs.content)

116

In [34]:
response.choices[0].logprobs.content[0].token

'First'

In [37]:
import tiktoken

In [38]:
# gpt_enc = tiktoken.get_encoding("cl100k_base")
gpt_enc = tiktoken.encoding_for_model("gpt-3.5-turbo-0613")

In [41]:
gpt_output = df['CoT Response'].tolist()[0]
encoded_output = gpt_enc.encode(gpt_output)

In [42]:
len(encoded_output)

116

In [44]:
steps = df['CoT Steps'].tolist()[0]

In [50]:
total_tokens = 0
for i, step in enumerate(steps):
    encoded_step = gpt_enc.encode(step)
    print(f"step {i}: ", len(encoded_step), total_tokens)
    print("step: ", step)
    total_tokens += len(encoded_step)
print("total tokens: ", total_tokens)

step 0:  56 0
step:  First, let's find out how many clips Natalia sold in May. We know that she sold half as many clips in May as she did in April. So, if she sold 48 clips in April, she sold 48/2 = 24 clips in May.
step 1:  44 56
step:  To find out how many clips Natalia sold altogether in April and May, we add the number of clips she sold in April to the number of clips she sold in May.   48 + 24 = 72
step 2:  16 100
step:  Therefore, Natalia sold a total of 72 clips in April and May.
total tokens:  116


In [51]:
response.choices[0].logprobs.content[100].token

'Therefore'

In [69]:
from collections import namedtuple
TokenLogProb = namedtuple('TokenLogProb', ['token', 'logprob'])

tokens_and_logprobs = [TokenLogProb(entry.token, entry.logprob) for entry in response.choices[0].logprobs.content]


In [60]:
len(tokens_and_logprobs)

116

In [61]:
row = df.iloc[0]

In [70]:
split_indices = []
total_tokens = 0
for i, step in enumerate(row['CoT Steps']):
    print("cumulative tokens: ", total_tokens)
    encoded_step = gpt_enc.encode(step)


    print(f"step {i}: ", len(encoded_step))
    print("step: ", step)
    start_index = total_tokens
    print("step starting token: ", tokens_and_logprobs[start_index].token)
    
    total_tokens += len(encoded_step)
    end_index = total_tokens - 1
    split_indices.append((start_index, end_index))

    print("step ending token: ", tokens_and_logprobs[end_index].token)
print("total tokens: ", total_tokens)

cumulative tokens:  0
step 0:  56
step:  First, let's find out how many clips Natalia sold in May. We know that she sold half as many clips in May as she did in April. So, if she sold 48 clips in April, she sold 48/2 = 24 clips in May.
step starting token:  First
step ending token:  .


cumulative tokens:  56
step 1:  44
step:  To find out how many clips Natalia sold altogether in April and May, we add the number of clips she sold in April to the number of clips she sold in May.   48 + 24 = 72
step starting token:  To
step ending token:  


cumulative tokens:  100
step 2:  16
step:  Therefore, Natalia sold a total of 72 clips in April and May.
step starting token:  Therefore
step ending token:  .
total tokens:  116


In [71]:
split_indices

[(0, 55), (56, 99), (100, 115)]

In [72]:
def get_split_indices(cot_tokens_and_logprobs, cot_steps, tokenizer):
    split_indices = []
    token_sum = 0
    
    for step in cot_steps:
        start_index = token_sum
        encoded_step = tokenizer.encode(step)
        token_sum += len(encoded_step)
        end_index = token_sum - 1
        split_indices.append((start_index, end_index))
    return split_indices 

In [73]:
split_indices = get_split_indices(tokens_and_logprobs, row['CoT Steps'], gpt_enc)

[(0, 55), (56, 99), (100, 115)]

In [76]:
start_index, end_index = split_indices[1]
start_index, end_index

(56, 99)

In [80]:
cur_logprobs = np.array([tokens_and_logprobs[i].logprob for i in range(start_index, end_index + 1)])
cur_logprobs.mean()

-0.15604184993850906

In [81]:
log_probs_arr = [entry.logprob for entry in response.choices[0].logprobs.content]
len(log_probs_arr)

116

Michael Method

In [92]:
def get_segment_ends(cot_steps, tokenizer, expected_total):
    segment_ends = []
    token_sum = 0
    
    for step in cot_steps:
        encoded_step = tokenizer.encode(step)
        token_sum += len(encoded_step)
        segment_ends.append(token_sum)
    assert token_sum == expected_total, f"Wrong number of tokens in CoT Steps: expected {expected_total}, got {token_sum}"
    return segment_ends

In [98]:
log_probs_arr = np.array([entry.logprob for entry in response.choices[0].logprobs.content]) # length 116
segment_ends = get_segment_ends(row['CoT Steps'], gpt_enc, len(tokens_and_logprobs)) # [56, 100, 116]
start = 0
segment_means = []

for end in segment_ends:
    segment_log_probs = log_probs_arr[start : end]
    segment_means.append(segment_log_probs.mean())
    print(f'Mean log prob for segment from {start} to {end}: {segment_log_probs.mean()}')
    start = end

Mean log prob for segment from 0 to 56: -0.12730754615481965
Mean log prob for segment from 56 to 100: -0.15604184993850906
Mean log prob for segment from 100 to 116: -0.0515566346777875
